In [4]:
import pandas as pd
from IPython.display import display

# Load the CSV
df_ftpt = pd.read_csv(
    r"C:\Users\jagod\project_group_2_fds_202526\data\07_part_time\raw\part_time.csv",
    low_memory=False
)

# Keep only the necessary columns
df_ftpt = df_ftpt[["Reference area", "TIME_PERIOD", "Sex", "MEASURE", "OBS_VALUE"]]

# Rename columns for clarity
df_ftpt.rename(columns={
    "Reference area": "Country",
    "TIME_PERIOD": "Year",
    "Sex": "Gender",
    "MEASURE": "Employment_Type",  # Full-time or Part-time
    "OBS_VALUE": "Employment_Rate"
}, inplace=True)

# Convert Employment_Rate to numeric
df_ftpt["Employment_Rate"] = pd.to_numeric(df_ftpt["Employment_Rate"], errors="coerce")

# Filter only Male and Female (exclude totals and aggregates)
df_ftpt = df_ftpt[df_ftpt["Gender"].isin(["Male", "Female"])]
exclude = ["OECD", "G20", "European Union (25 countries)"]
df_ftpt = df_ftpt[~df_ftpt["Country"].isin(exclude)]

# Pivot table to have Male/Female in separate columns
df_pivot = df_ftpt.pivot_table(
    index=["Country", "Year", "Employment_Type"],
    columns="Gender",
    values="Employment_Rate"
).reset_index()

# Flatten columns
df_pivot.columns.name = None

# Add Female-to-Male ratio and Female share
df_pivot["F_to_M_ratio"] = df_pivot["Female"] / df_pivot["Male"]
df_pivot["Female_Share"] = df_pivot["Female"] / (df_pivot["Female"] + df_pivot["Male"])

# Sort alphabetically by Country and by Employment Type
df_pivot = df_pivot.sort_values(by=["Country", "Year", "Employment_Type"]).reset_index(drop=True)

# Display as a clean table
pd.set_option("display.float_format", "{:.2f}".format)
display(df_pivot)


,Country,Year,Employment_Type,Female,Male,F_to_M_ratio,Female_Share
0,Argentina,2016,EMP,50.00,50.00,1.00,0.50
1,Argentina,2016,EMP_PT,74.17,25.83,2.87,0.74
2,Argentina,2017,EMP,50.00,50.00,1.00,0.50
3,Argentina,2017,EMP_PT,70.09,29.91,2.34,0.70
4,Argentina,2018,EMP,50.00,50.00,1.00,0.50
...,...,...,...,...,...,...,...
1441,United Kingdom,2022,EMP_PT,76.60,23.40,3.27,0.77
1442,United Kingdom,2023,EMP,50.00,50.00,1.00,0.50
1443,United Kingdom,2023,EMP_PT,75.62,24.38,3.10,0.76
1444,United Kingdom,2024,EMP,50.00,50.00,1.00,0.50
